In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from urllib.request import urlopen

In [ ]:
url = "https://fbref.com/en/matches/"
headers = {"User-Agent": "Mozilla/5.0"}
fixtures = []

try:
    response = requests.get(url, headers=headers, timeout=10)
    response.raise_for_status()  # raise error for bad responses
    soup = BeautifulSoup(response.content, "html.parser")
except requests.RequestException as e:
    print(f"Error fetching {url}: {e}")
    fixtures = []


In [2]:
# Loop through each competition section
for header in soup.find_all("h2"):
    competition = header.get_text(strip=True).replace(" Scores & Fixtures", "")
    table = header.find_next("table")
    if not table:
        continue

    # Extract headers from the table
    try:
        headers_row = [th.get_text(strip=True) for th in table.find("thead").find_all("th")]
    except AttributeError:
        print(f"No headers found for {competition}, skipping...")
        continue

    tbody = table.find("tbody")
    if not tbody:
        continue

    for row in tbody.find_all("tr"):
        cells = row.find_all("td")
        if not cells:
            continue  # skip header/separator rows

        row_data = {"Competition": competition}

        # Map cells to their respective headers
        for i, cell in enumerate(cells):
            col_name = headers_row[i+1] if i+1 < len(headers_row) else f"Col_{i}"
            row_data[col_name] = cell.get_text(strip=True) if cell else None

            # Capture match link if available
            link = cell.find("a")
            if link and "href" in link.attrs:
                row_data[f"{col_name}_URL"] = "https://fbref.com" + link["href"]

        fixtures.append(row_data)

# Preview results
print(f"Collected {len(fixtures)} fixtures")
if fixtures:
    for fix in fixtures[:5]:
        print(fix)

Collected 35 fixtures
{'Competition': 'engChampionship', 'Wk': '6', 'Time': '20:00', 'Home': 'Millwall', 'Home_URL': 'https://fbref.com/en/squads/e3c537a1/Millwall-Stats', 'xG': '0.7', 'Score': '1–0', 'Score_URL': 'https://fbref.com/en/matches/a0d4b2d4/Millwall-Watford-September-22-2025-Championship', 'Away': 'Watford', 'Away_URL': 'https://fbref.com/en/squads/2abfe087/Watford-Stats', 'Attendance': '14,586', 'Venue': 'The Den', 'Referee': 'Joshua Smith', 'Match Report': 'Match Report', 'Match Report_URL': 'https://fbref.com/en/matches/a0d4b2d4/Millwall-Watford-September-22-2025-Championship', 'Notes': ''}
{'Competition': 'esSegunda División', 'Wk': '6', 'Time': '20:30', 'Home': 'Burgos', 'Home_URL': 'https://fbref.com/en/squads/d9fac124/Burgos-Stats', 'Score': '1–1', 'Score_URL': 'https://fbref.com/en/matches/0dea0916/Burgos-Granada-September-22-2025-Segunda-Division', 'Away': 'Granada', 'Away_URL': 'https://fbref.com/en/squads/a0435291/Granada-Stats', 'Attendance': '8,887', 'Venue': '